# Collector
From its literal meaning, we can easily know that the Collector in Tianshou is used to collect training data. More specifically, the Collector controls the interaction between Policy (agent) and the environment. It also helps save the interaction data into the ReplayBuffer and returns episode statistics.

<center>
<img src=../_static/images/structure.svg></img>
</center>



## Usages
Collector can be used both for training (data collecting) and evaluation in Tianshou.

### Policy evaluation
We need to evaluate our trained policy from time to time in DRL experiments. Collector can help us with this.

First we have to initialize a Collector with an (vectorized) environment and a given policy (agent).

In [ ]:
%%capture

import gymnasium as gym
import torch

from tianshou.data import Collector, CollectStats, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.algorithm import PGPolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import Actor

In [ ]:
env = gym.make("CartPole-v1")
test_envs = DummyVectorEnv([lambda: gym.make("CartPole-v1") for _ in range(2)])

# model
assert env.observation_space.shape is not None  # for mypy
net = Net(
    env.observation_space.shape,
    hidden_sizes=[
        16,
    ],
)

assert isinstance(env.action_space, gym.spaces.Discrete)  # for mypy
actor = Actor(net, env.action_space.n)
optim = torch.optim.Adam(actor.parameters(), lr=0.0003)

policy: PGPolicy = PGPolicy(
    actor=actor,
    optim=optim,
    dist_fn=torch.distributions.Categorical,
    action_space=env.action_space,
    action_scaling=False,
)
test_collector = Collector[CollectStats](policy, test_envs)

Now we would like to collect 9 episodes of data to test how our initialized Policy performs.

In [ ]:
collect_result = test_collector.collect(reset_before_collect=True, n_episode=9)

collect_result.pprint_asdict()

Now we wonder what is the performance of a random policy.

In [ ]:
# Reset the collector
collect_result = test_collector.collect(reset_before_collect=True, n_episode=9, random=True)

collect_result.pprint_asdict()

It seems like an initialized policy performs even worse than a random policy without any training.

### Data Collecting
Data collecting is mostly used during training, when we need to store the collected data in a ReplayBuffer.

In [ ]:
train_env_num = 4
buffer_size = 100
train_envs = DummyVectorEnv([lambda: gym.make("CartPole-v1") for _ in range(train_env_num)])
replayBuffer = VectorReplayBuffer(buffer_size, train_env_num)

train_collector = Collector[CollectStats](policy, train_envs, replayBuffer)

Now we can collect 50 steps of data, which will be automatically saved in the replay buffer. You can still choose to collect a certain number of episodes rather than steps. Try it yourself.

In [ ]:
train_collector.reset()
replayBuffer.reset()

print(f"Replay buffer before collecting is empty, and has length={len(replayBuffer)} \n")
n_step = 50
collect_result = train_collector.collect(n_step=n_step)
print(
    f"Replay buffer after collecting {n_step} steps has length={len(replayBuffer)}.\n"
    f"This may exceed n_step when it is not a multiple of train_env_num because of vectorization.\n",
)
collect_result.pprint_asdict()

Sample some data from the replay buffer.

In [ ]:
replayBuffer.sample(10)

## Further Reading
The above collector actually collects 52 data at a time because 52 % 4 = 0. There is one asynchronous collector which allows you collect exactly 50 steps. Check the [documentation](https://tianshou.org/en/master/03_api/data/collector.html#tianshou.data.collector.AsyncCollector) for details.